In [1]:
df = spark.sql("SELECT CustomerID as CustomerKey,FirstName,LastName ,FullName ,DateInserted FROM Sales_Silver.Customers")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimCustomer")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 3, Finished, Available)

In [2]:
%%sql
--nb_Employee_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpEmployee
AS
SELECT   EmployeeID EmployeeKey
            ,ManagerID
            ,FirstName
            ,LastName
            ,FullName
            ,JobTitle
            ,OrganizationLevel
            ,MaritalStatus
            ,Gender
            ,Territory
            ,Country
            ,Group
FROM Sales_Silver.Employees

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
df = spark.sql("SELECT * FROM tmpEmployee")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimEmployee")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 5, Finished, Available)

In [4]:
%%sql
--nb_DimVendor_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpVendors
AS
SELECT       VendorID as VendorKey
            ,VendorName
            ,AccountNumber
            ,CreditRating
            ,ActiveFlag
            ,DateInserted
FROM Sales_Silver.Vendors

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
df = spark.sql("SELECT * FROM tmpVendors")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimVendor")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 7, Finished, Available)

In [6]:
%%sql
--nb_DimProductSubCategories_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProductSubCategory
AS
SELECT      SubCategoryID as SubCategoryKey
            ,CategoryID as CategoryKey
            ,SubCategoryName
FROM Sales_Silver.ProductSubCategories

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 8, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
df = spark.sql("SELECT * FROM tmpProductSubCategory")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProductSubCategory")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 9, Finished, Available)

In [8]:
%%sql
--nb_DimProductCategorie_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProductCategory
AS
SELECT       CategoryID as CategoryKey
            ,CategoryName
            ,DateInserted
FROM Sales_Silver.ProductCategories 

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 10, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
df = spark.sql("SELECT * FROM tmpProductCategory")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProductCategory")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 11, Finished, Available)

In [10]:
%%sql
--nb_DimProduct_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProducts
AS
SELECT    ProductID as ProductKey
            ,ProductNumber
            ,ProductName
            ,ModelName
            ,MakeFlag
            ,StandardCost
            ,ListPrice
            ,SubCategoryID
            ,DateInserted
FROM Sales_Silver.Products    

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 12, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
df = spark.sql("SELECT * FROM tmpProducts")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProduct")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 13, Finished, Available)

In [12]:
%%sql
--nb_FactSales_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpSales
AS
SELECT      SalesOrderID AS SalesOrderKey
            ,ifnull(e.EmployeeKey,-1) as EmployeeKey
            ,ifnull(c.CustomerKey,-1) as CustomerKey
            ,ifnull(p.ProductKey,-1) as ProductKey
            ,ifnull(vp.VendorID,-1) as VendorKey
            ,ifnull(psc.CategoryKey,-1) as CategoryKey
            ,ifnull(psc.SubCategoryKey,-1) as SubCategoryKey                       
            ,SalesOrderDetailID
            ,OrderDate
            ,DueDate
            ,ShipDate
            ,SubTotal
            ,TaxAmt
            ,Freight
            ,TotalDue
            ,OrderQty
            ,UnitPrice
            ,UnitPriceDiscount
            ,LineTotal
            ,o.DateInserted
FROM Sales_Silver.Orders o
LEFT JOIN Sales_Gold.DimCustomer c ON c.CustomerKey =o.CustomerID
LEFT JOIN Sales_Gold.DimEmployee e ON e.EmployeeKey= o.EmployeeID
LEFT JOIN Sales_Gold.DimProduct p ON p.ProductKey = o.ProductID
LEFT JOIN 
(
SELECT
   ProductID,
   VendorID
from 
Sales_Silver.VendorProduct
group by 
   ProductID,
   VendorID
) vp ON vp.ProductID = p.ProductKey
LEFT JOIN Sales_Gold.DimProductSubCategory psc ON psc.SubCategoryKey=p.SubCategoryID

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 14, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
df = spark.sql("SELECT * FROM tmpSales")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/FactSales")

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 15, Finished, Available)

In [14]:
display(df)

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8c96d2c0-b04a-4be4-a351-9d86c3aad49d)

In [15]:
df = spark.sql("SELECT * FROM Sales_Gold.DimProduct LIMIT 1000")
display(df)

StatementMeta(, 5d5151a3-ae99-4f9d-9480-04246b41fd7f, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, b1aea1a0-8dc8-417c-b68c-14be5e51e701)